# 由于在boost模型建模的时候发现序列特征是最强的，与其手动调试不如搭建一个网络自动学习

In [1]:
import os
import matplotlib.pyplot as plt
import glob
import gc
from math import *
import warnings
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from keras.layers import Dense, Dropout, LSTM,Reshape, GRU,Conv1D, Conv2D,Flatten,Permute, multiply,BatchNormalization, Activation, AveragePooling1D, GlobalAveragePooling1D, Lambda, Input, Concatenate, Add, UpSampling1D, Multiply
from keras.models import Model
from keras.objectives import mean_squared_error
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau,LearningRateScheduler
from keras.initializers import random_normal
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import Callback
from keras_self_attention import SeqSelfAttention
from sklearn.metrics import cohen_kappa_score, f1_score,roc_auc_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,normalize

warnings.filterwarnings('ignore')

C:\Users\Ksama\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ksama\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ksama\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\K

In [2]:
def reduce_mem(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [3]:
train_df = pd.read_csv('../data/train.csv',low_memory=False)
test_df = pd.read_csv('../data/test.csv',low_memory=False)

#根据“第二次打比赛”队伍的发现进行简单的数据清洗
train_df= train_df[(train_df['q']>0)]
train_df = train_df[train_df['t']>-900]
train_df = train_df[train_df['t']<1850]

train_num = len(train_df)
data = pd.concat([train_df, test_df], ignore_index=True)
event = pd.read_csv('../data/event.csv')
data = pd.merge(data, event, on='event_id', how='left')
del train_df,test_df,event
gc.collect()

DeepCTR version 0.7.4 detected. Your version is 0.7.3.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.7.4


7

In [4]:
#非序列特征：
#由于开始做深度模型的时间已经很晚了，故把精力放在序列的构造上，并没有过多的进行非序列特征的构造
data['x_cmc'] = (data['x']-data['xcmc'])
data['y_cmc'] = (data['y']-data['ycmc'])
data['dis'] = np.sqrt(data['x_cmc']**2+data['y_cmc']**2)

# t ,dis统计特征
data['event_id_t_min'] = data.groupby('event_id')['t'].transform('min')
data['event_id_t_max'] = data.groupby('event_id')['t'].transform('max')
data['event_id_t_median'] = data.groupby('event_id')['t'].transform('median')
data['event_id_t_mean'] = data.groupby('event_id')['t'].transform('mean')

data['event_id_dis_min'] = data.groupby('event_id')['dis'].transform('min')
data['event_id_dis_max'] = data.groupby('event_id')['dis'].transform('max')
data['event_id_dis_median'] = data.groupby('event_id')['dis'].transform('median')
data['event_id_dis_mean'] = data.groupby('event_id')['dis'].transform('mean')

# t,dis"偏移"
data['t_min_diff'] = data['t'] - data['event_id_t_min']
data['t_max_diff'] = data['event_id_t_max'] - data['t']
data['t_median_diff'] = data['event_id_t_median'] - data['t']
data['t_mean_diff'] = data['event_id_t_mean'] - data['t']

data['dis_min_diff'] = data['dis'] - data['event_id_dis_min']
data['dis_max_diff'] = data['event_id_dis_max'] - data['dis']
data['dis_median_diff'] = data['event_id_dis_median'] - data['dis']
data['dis_mean_diff'] = data['event_id_dis_mean'] - data['dis']

In [5]:
#归一化
sclaer = StandardScaler()
scale_fea = [x for x in data.columns if x not in ['event_id','hit_id','flag','z','level_0','index']]
data[scale_fea] = sclaer.fit_transform(data[scale_fea].values)

In [6]:
#构造序列特征，用做wavenet的输入
seq_len = 64
deleta = 2
gap = int(seq_len/2)
temp = data.sort_values(['q'])

timing_cols=[]

for i in tqdm(range(seq_len)):
    data['t_gap_{}'.format(i)] = temp['t'].shift(deleta*(i-gap)).fillna(0)
    timing_cols += ['t_gap_{}'.format(i)]
print(data.shape)
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [01:03<00:00,  1.00it/s]


(13382309, 99)
10209.89 Mb, 2641.81 Mb (74.12 %)


In [7]:
temp = data.sort_values(['event_id','q'])

q_cols=[]

for i in tqdm(range(seq_len)):
    data['q_gap_{}'.format(i)] = temp['q'].shift(deleta*(i - gap)).fillna(0)
    q_cols += ['q_gap_{}'.format(i)]
print(data.shape)
del temp
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [00:48<00:00,  1.33it/s]


(13382309, 163)
4275.39 Mb, 4275.39 Mb (0.00 %)


In [8]:
temp = data.sort_values(['terror','q'])

teq_cols=[]

for i in tqdm(range(seq_len)):
    data['tq_gap_{}'.format(i)] = temp['q'].shift(deleta*(i - gap)).fillna(0)
    teq_cols += ['tq_gap_{}'.format(i)]
print(data.shape)
del temp
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [00:49<00:00,  1.30it/s]


(13382309, 227)
5908.97 Mb, 5908.97 Mb (0.00 %)


In [9]:
temp = data.sort_values(['hit_id'])

ht_cols=[]

for i in tqdm(range(seq_len)):
    data['ht_gap_{}'.format(i)] = temp['t'].shift(deleta*(i - gap)).fillna(0)
    ht_cols += ['ht_gap_{}'.format(i)]
print(data.shape)
del temp
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [00:47<00:00,  1.34it/s]


(13382309, 291)
7542.56 Mb, 7542.56 Mb (0.00 %)


In [10]:
temp = data.sort_values(['terror','q'])

tet_cols=[]

for i in tqdm(range(seq_len)):
    data['tet_gap_{}'.format(i)] = temp['t'].shift(deleta*(i - gap)).fillna(0)
    tet_cols += ['tet_gap_{}'.format(i)]
print(data.shape)
del temp
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [00:52<00:00,  1.23it/s]


(13382309, 355)
9176.14 Mb, 9176.14 Mb (0.00 %)


In [11]:
temp = data.sort_values(['event_id','q'])

qdis_cols=[]

for i in tqdm(range(seq_len)):
    data['qdis_gap_{}'.format(i)] = temp['dis'].shift(deleta*(i - gap)).fillna(0)
    qdis_cols += ['qdis_gap_{}'.format(i)]
print(data.shape)
del temp
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [00:49<00:00,  1.29it/s]


(13382309, 419)
10809.72 Mb, 10809.72 Mb (0.00 %)


In [12]:
temp = data.sort_values(['terror','q'])

tqdis_cols=[]

for i in tqdm(range(seq_len)):
    data['tqdis_gap_{}'.format(i)] = temp['dis'].shift(deleta*(i - gap)).fillna(0)
    tqdis_cols += ['tqdis_gap_{}'.format(i)]
print(data.shape)
del temp
data = reduce_mem(data)

100%|██████████████████████████████████████████████████████████████| 64/64 [00:52<00:00,  1.21it/s]


(13382309, 483)
12443.31 Mb, 12443.31 Mb (0.00 %)


In [13]:
train_df = data[:train_num].reset_index()
test_df = data[train_num:].reset_index()
train_df=train_df.sample(frac=1.0)
train_df=train_df.reset_index(drop=True)
del data
gc.collect()

10

In [14]:
train_data_1 = np.array(train_df[timing_cols]).reshape(len(train_df), seq_len, 1)  
train_data_2 = np.array(train_df[q_cols]).reshape(len(train_df), seq_len, 1)  
train_data_3 = np.array(train_df[teq_cols]).reshape(len(train_df), seq_len, 1)  
train_data_4 = np.array(train_df[ht_cols]).reshape(len(train_df), seq_len, 1)  
train_data_5 = np.array(train_df[tet_cols]).reshape(len(train_df), seq_len, 1)  
train_data_6 = np.array(train_df[qdis_cols]).reshape(len(train_df), seq_len, 1)  
train_data_7 = np.array(train_df[tqdis_cols]).reshape(len(train_df), seq_len, 1)  

train_data_seq = np.concatenate([train_data_1,train_data_2,train_data_3,train_data_4,train_data_5,train_data_6,train_data_7],axis=2)

train_label = train_df['flag'].values
train_data_sideinfo = train_df[scale_fea].values  
del train_data_1,train_data_2,train_data_3,train_data_4,train_data_5,train_data_6,train_data_7

test_data_1 = np.array(test_df[timing_cols]).reshape(len(test_df), seq_len, 1)  
test_data_2 = np.array(test_df[q_cols]).reshape(len(test_df), seq_len, 1)  
test_data_3 = np.array(test_df[teq_cols]).reshape(len(test_df), seq_len, 1)  
test_data_4 = np.array(test_df[ht_cols]).reshape(len(test_df), seq_len, 1)  
test_data_5 = np.array(test_df[tet_cols]).reshape(len(test_df), seq_len, 1)  
test_data_6 = np.array(test_df[qdis_cols]).reshape(len(test_df), seq_len, 1)  
test_data_7 = np.array(test_df[tqdis_cols]).reshape(len(test_df), seq_len, 1)  

test_data_seq = np.concatenate([test_data_1,test_data_2,test_data_3,test_data_4,test_data_5,test_data_6,test_data_7],axis=2)

test_data_sideinfo = test_df[scale_fea].values 

del test_data_1,test_data_2,test_data_3,test_data_4,test_data_5,test_data_6,test_data_7
gc.collect()
print(train_data_seq.shape,train_data_sideinfo.shape)

(9295798, 64, 7) (9295798, 31)


In [15]:
# 训练集验证集按照 7:3比例分割
train_size = int(len(train_data_lstm) * 0.7)
# 训练集、验证集 seq特征
X_train_seq, X_validate_seq = train_data_seq[:train_size], train_data_seq[train_size:]
label_train, label_validate = train_label[:train_size], train_label[train_size:]

# 训练集、验证集 dnn的特征
X_train_side, X_validate_side = train_data_sideinfo[:train_size], train_data_sideinfo[train_size:]

In [17]:
#采用CyclicLR学习率衰减
class CyclicLR(tf.keras.callbacks.Callback):

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** (x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(
                self.clr_iterations)

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, epoch, logs=None):

        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        K.set_value(self.model.optimizer.lr, self.clr())


In [16]:
#wavenet的组成部分
def wave_block(x,filters,kernel_size,n):
    dilation_rates = [2**i for i in range(n)]
    x = Conv1D(filters=filters,
                kernel_size=1, 
                padding='same')(x)
    res_x = x
    for dilation_rate in dilation_rates:
        tanh_out = Conv1D(filters=filters,
                kernel_size=kernel_size, 
                padding='same',
                activation = 'tanh',
                dilation_rate=dilation_rate)(x)
        sigm_out = Conv1D(filters=filters,
                kernel_size=kernel_size, 
                padding='same',
                activation = 'sigmoid',
                dilation_rate=dilation_rate)(x)
        x = Multiply()([tanh_out,sigm_out])
        x = Conv1D(filters = filters,
                       kernel_size = 1,
                      padding='same')(x)

        res_x = Add()([res_x,x])
    return res_x

#attention机制
def attention_3d_block(inputs, seq_len=21):
    a = Permute((2, 1))(inputs)
    a = Dense(seq_len, activation='softmax')(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [18]:
def nn_model(seq_len,fea_len,att=True):
    
    seq_input =  Input((seq_len,7), name='seq_input')
    nn_input = Input((fea_len,), name='nn_input')
     
#wavenet   
    x = wave_block(seq_input,8,3,3)
    x = wave_block(x,  12,3,2)
    x = wave_block(x,  16,3,2)
    seq_out = wave_block(x, 20,3,1)

#合并  
    seq_out = Dense(64,kernel_regularizer=regularizers.l2(0.01))(seq_out)
    attention_mul = attention_3d_block(seq_out, seq_len=seq_len)
    seq_out = Lambda(lambda x: K.sum(x, axis=1))(attention_mul)
    
#dnn input
    dnn_input = Concatenate()([seq_out,nn_input])

    x = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dnn_input)
    x = BatchNormalization()(x)
    
    x = Dense(128,activation='relu')(x)
    x = BatchNormalization()(x)
    
    out = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=[seq_input, nn_input], outputs=out)
    
    return model

In [19]:
model = nn_model(seq_len=seq_len,fea_len=len(scale_fea))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq_input (InputLayer)          (None, 64, 7)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 64, 8)        64          seq_input[0][0]                  
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 64, 8)        200         conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 64, 8)        200         conv1d_1[0][0]                   
__________________________________________________________________________________________________
multiply_1

In [21]:
#输出train&valid的auc
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [23]:
def lrs(epoch):
    if epoch<10:
        lr = learning_rate
    elif epoch<20:
        lr = learning_rate/10
    elif epoch<40:
        lr = learning_rate/100
    elif epoch<70:
        lr = learning_rate/500
    else:
        lr = learning_rate/1000
    return lr

lr_schedule = LearningRateScheduler(lrs)

In [25]:
#由于机器配置原因导致训练时间过久，故没有做cv，加上cv应该还会有提升
learning_rate = 0.0006
n_epoch=200
batch_size = 2048
cb_clr = CyclicLR(base_lr=1e-7, max_lr = 1e-4, step_size= int(1.0*(test_data_sideinfo.shape[0])/(batch_size*4)) , mode='exp_range', gamma=1.0, scale_fn=None, scale_mode='cycle')
plateau = ReduceLROnPlateau(monitor="val_auroc", verbose=1, mode='max', factor=0.3, patience=5)
early_stopping = EarlyStopping(monitor='val_auroc', patience=10, mode='max')
opt = Adam(lr=learning_rate)
model.compile(
              loss='binary_crossentropy',
              optimizer=opt,
             metrics=['accuracy',auroc])

# train the model
print("[INFO] training model...")
model.fit(
    [X_train_seq,X_train_side], label_train,
    validation_data=([X_validate_seq,X_validate_side], label_validate),
    callbacks=[early_stopping,lr_schedule,cb_clr,plateau],shuffle=True,
    epochs=n_epoch, batch_size=batch_size)

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


[INFO] training model...
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Train on 6507058 samples, validate on 2788740 samples
Epoch 1/200
6507058/6507058 [==============================] - 226s 35us/step - loss: 0.7023 - acc: 0.9628 - auroc: 0.9874 - val_loss: 0.1550 - val_acc: 0.9830 - val_auroc: 0.9983
Epoch 2/200
6507058/6507058 [==============================] - 202s 31us/step - loss: 0.0797 - acc: 0.9865 - auroc: 0.9989 - val_loss: 0.0508 - val_acc: 0.9882 - val_auroc: 0.9991
Epoch 3/200
6507058/6507058 [==============================] - 203s 31us/step - loss: 0.0431 - acc: 0.9888 - auroc: 0.9993 - val_loss: 0.0401 - val_acc: 0.9886 - val_auroc: 0.9992
Epoch 4/200
6507058/6507058 [==============================] - 201s 31us/step - loss: 0.0363 - acc: 0.9894 - auroc: 0.9993 - val_loss: 0.0344 - val_acc: 0.9896 - val_auroc: 0.9994
Epoch 5/200
6507058/6507058 [==============================] - 198s 30us/step - loss: 0.0333 - acc: 0.9898 - auroc: 0.9994 - val_loss: 0.0310 - val_acc: 0.9903 - val_auroc: 0.9994
Epoch 6/200
6507058/6507058 [=================

Epoch 80/200
6507058/6507058 [==============================] - 203s 31us/step - loss: 0.0246 - acc: 0.9915 - auroc: 0.9996 - val_loss: 0.0247 - val_acc: 0.9915 - val_auroc: 0.9996
Epoch 81/200
6507058/6507058 [==============================] - 203s 31us/step - loss: 0.0246 - acc: 0.9914 - auroc: 0.9996 - val_loss: 0.0254 - val_acc: 0.9913 - val_auroc: 0.9996
Epoch 82/200
6507058/6507058 [==============================] - 203s 31us/step - loss: 0.0246 - acc: 0.9915 - auroc: 0.9996 - val_loss: 0.0258 - val_acc: 0.9911 - val_auroc: 0.9996
Epoch 83/200
6507058/6507058 [==============================] - 203s 31us/step - loss: 0.0246 - acc: 0.9915 - auroc: 0.9996 - val_loss: 0.0248 - val_acc: 0.9914 - val_auroc: 0.9996
Epoch 84/200
6506496/6507058 [============================>.] - ETA: 0s - loss: 0.0246 - acc: 0.9915 - auroc: 0.9996
Epoch 00084: reducing learning rate to 1.4743373412784422e-06.
6507058/6507058 [==============================] - 202s 31us/step - loss: 0.0246 - acc: 0.9915 -

In [26]:
pred_valid = model.predict([X_validate_seq,X_validate_side],batch_size=batch_size)
pred = model.predict([test_data_seq,test_data_sideinfo],batch_size=batch_size)
print("AUC: {}".format(roc_auc_score(label_validate,pred_valid)))

AUC: 0.9996128383079947


In [70]:
#保存预测概率用于融合
tt = pd.read_csv('../data/test.csv')
sub = pd.DataFrame()
tt['flag_pre'] =pred
tt.loc[tt['t']>1850,'flag_pre']=1
tt.loc[tt['t']<-900,'flag_pre']=0
tt.loc[tt['q']<0,'flag_pre']=1
sub['hit_id']=tt['hit_id']
sub['flag_pred'] = tt['flag_pre']
sub['event_id'] = tt['event_id']
sub.to_csv('../result/wavenet_prob.csv',index=False)